### Dependencies

In [1]:
# Use these commands to install required dependencies if necessary.

# !pip install pandas findspark py4j seaborn numpy
# !pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio===0.11.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
# !pip install transformers
# !pip install tqdm
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

# Use this command if the above installation of PyTorch fails.

# !pip install torch torchvision torchaudio

#### Code To Ignore Warning Messages

In [3]:
#Doesn't seem to work here properly

import warnings
warnings.filterwarnings('ignore')

warnings.filterwarnings(action='once')

In [5]:
%%javascript
(function(on) {
const e=$( "<a>Setup failed</a>" );
const ns="js_jupyter_suppress_warnings";
var cssrules=$("#"+ns);
if(!cssrules.length) cssrules = $("<style id='"+ns+"' type='text/css'>div.output_stderr { } </style>").appendTo("head");
e.click(function() {
    var s='Showing';  
    cssrules.empty()
    if(on) {
        s='Hiding';
        cssrules.append("div.output_stderr, div[data-mime-type*='.stderr'] { display:none; }");
    }
    e.text(s+' warnings (click to toggle)');
    on=!on;
}).click();
$(element).append(e);
})(true);

<IPython.core.display.Javascript object>

In [6]:
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

### Imports

In [7]:
import pkg_resources
pkg_resources.require("torch==1.11.0")

import torch

import pandas as pd
import numpy as np

from time import sleep
import json
import os

from collections import namedtuple
import sqlite3

from tqdm.notebook import tqdm

from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [9]:
import findspark

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.functions import desc
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [8]:
pd.set_option('display.max_colwidth', None)

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### BERTweet Model Set-Up

In [11]:
model_type = "finiteautomata/bertweet-base-sentiment-analysis"

tokenizer = AutoTokenizer.from_pretrained(model_type)
model = AutoModelForSequenceClassification.from_pretrained(model_type)

emoji is not installed, thus not converting emoticons or emojis into text. Please install emoji: pip3 install emoji


In [1]:
def sentiment_score(tweet):
    """To return the sentiment score of a Tweet as analysed by BERTweet. """
    tokens = tokenizer.encode(tweet, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))-1

### Spark Set-Up

In [ ]:
# Constants for FILE PATHS

SPARK_PATH = '/home/venky/spark-3.2.1-bin-hadoop3.2'

In [10]:
findspark.init(SPARK_PATH)
findspark.add_packages("org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1")    #Required dependency

In [13]:
spark = SparkSession.builder.appName("FYP").getOrCreate()
spark

In [14]:
#Run this only once, restart kernel if errors
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=FYP>

In [15]:
TABLE_COUNT = 0
IN_MEM_TABLENAME = "TweetData"
SQLITE_TABLENAME = "scored_tweets"
OFFSET = 0
TOPIC = "gaming-tweets"

In [16]:
def check_offset_status():
    connection = sqlite3.connect(os.path.join(os.getcwd(), f'../Database/cache.sqlite'))
    cursor = connection.cursor()

    query = f"SELECT offsetval FROM OFFSET_FINDER WHERE topic LIKE ?"

    rows = cursor.execute(query, [TOPIC]).fetchall()

    if rows:
        OFFSET = rows[0][0]
    else:
        insert_query = f"INSERT INTO OFFSET_FINDER VALUES(?, ?)"
        cursor.execute(insert_query, (TOPIC, 0))
        connection.commit()

    print({f"Starting Offset for {TOPIC}": OFFSET})

    cursor.close()
    connection.close()

{'Starting Offset for gaming-tweets': 0}


In [ ]:
check_offset_status()

In [17]:
# df = spark \
#   .readStream \
#   .format("kafka") \
#   .option("kafka.bootstrap.servers", "localhost:9092") \
#   .option("subscribe", TOPIC) \
#   .option("startingOffsets", f""" {{"{TOPIC}":{{"0":{OFFSET}}}}} """) \
#   .load()

# schema_str = "Data STRING"

# df = df.selectExpr("CAST(value AS STRING)")
# df = df.select(from_csv(col("value"),schema_str).alias("Table"))
# df = df.selectExpr("Table.*")
# df.printSchema()
# #option("truncate", "false")

In [18]:
# query = df.writeStream.trigger(processingTime='5 seconds').queryName(f"{IN_MEM_TABLENAME}{TABLE_COUNT}").format('memory').outputMode("append").start()


In [19]:
spark.sql('SHOW TABLES').show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [20]:
# sleep(10)

# tweet_dict_list = []

# value = spark.sql(f"SELECT * FROM {IN_MEM_TABLENAME}{TABLE_COUNT} LIMIT 10").collect()
# for row in value:
#     #print(row)
#     jsonCopy = json.loads(row["Data"])
#     jsonCopy['score'] = sentiment_score(jsonCopy['tweet'][:135])
#     tweet_dict_list.append(jsonCopy)
# pdd = pd.DataFrame(tweet_dict_list)

# query.awaitTermination(1)
# pdd.head(10)

In [21]:
# rdd = sc.parallelize(tweet_dict_list)

In [22]:
# %%time
# rdd.map(lambda row: (row['tweet'], row['score'])).toDF().toPandas().head(10)

In [23]:
# newrdd = rdd.map(lambda row: (row['category'], row['date'], row['count'], row['score']))
# newrdd.collect()

In [24]:
# nextrdd = newrdd.map(lambda tup: ((tup[0], tup[1]), (tup[2]*tup[3], tup[2]))).reduceByKey(lambda a, b: (a[0]+b[0], a[1]+b[1]))
# nextrdd.collect()

In [25]:
#write to db
# connection = sqlite3.connect(os.path.join(os.getcwd(), f'../Database/results.sqlite'))
# cursor = connection.cursor()

# drop_table = f'''
#             DROP TABLE IF EXISTS {SQLITE_TABLENAME};
#             '''

# cursor.execute(drop_table)


# create_table = f'''CREATE TABLE IF NOT EXISTS {SQLITE_TABLENAME} (
#                 category TEXT,
#                 date DATE,
#                 score INTEGER,
#                 count INTEGER,
#                 CONSTRAINT uniq_val PRIMARY KEY (category, date)
#                 );
#                 '''

# cursor.execute(create_table)

# insert_records = f'''INSERT INTO {SQLITE_TABLENAME} (category, date, score, count) VALUES(?, ?, ?, ?)
#                         ON CONFLICT(category, date) DO 
#                         UPDATE SET score = score + excluded.score, count = count + excluded.count 
#                         WHERE {SQLITE_TABLENAME}.category LIKE ? AND {SQLITE_TABLENAME}.date LIKE ? '''
    

# contents = []
# for row in nextrdd.collect():
#     contents.append((row[0][0], row[0][1], row[1][0], row[1][1], row[0][0], row[0][1]))
    
# try:
#     cursor.executemany(insert_records, contents)
#     connection.commit()

#     rows = cursor.execute(f"SELECT * FROM {SQLITE_TABLENAME}").fetchall()
#     for row in rows:
#         print(row)
# except sqlite3.Error as error:
#     print({error})
# finally:
#     cursor.close()
#     connection.close()

### Helper Methods 

In [ ]:
def init_df_table():
    """To initialize a Spark DataFrame with data ingested from Kafka. """
    
    df = spark \
      .readStream \
      .format("kafka") \
      .option("kafka.bootstrap.servers", "localhost:9092") \
      .option("subscribe", TOPIC) \
      .option("startingOffsets", f""" {{"{TOPIC}":{{"0":{OFFSET}}}}} """) \
      .load()

    schema_str = "Data STRING"

    df = df.selectExpr("CAST(value AS STRING)")
    df = df.select(from_csv(col("value"),schema_str).alias("Table"))
    df = df.selectExpr("Table.*")
    df.printSchema()

    query = df.writeStream \
                .trigger(processingTime='5 seconds') \
                .queryName(f"{IN_MEM_TABLENAME}{TABLE_COUNT}") \
                .format('memory') \
                .outputMode("append") \
                .start()
    
    spark.sql('SHOW TABLES').show()
    return query

In [ ]:
def delete_spark_sql_table():
    """To delete existing SparkSQL tables from memory. """
    
    connection = sqlite3.connect(os.path.join(os.getcwd(), f'../Database/results.sqlite'))
    cursor = connection.cursor()
    
    drop_table = f'''
            DROP TABLE IF EXISTS {SQLITE_TABLENAME};
            '''

    cursor.execute(drop_table)


    create_table = f'''CREATE TABLE IF NOT EXISTS {SQLITE_TABLENAME} (
                category TEXT,
                date DATE,
                score INTEGER,
                count INTEGER,
                CONSTRAINT uniq_val PRIMARY KEY (category, date)
                );
                '''

    cursor.execute(create_table)
    
    cursor.close()
    connection.close()

In [ ]:
def write_to_db(rdd):
    """To write a SparkSQL table to permanent storage. """
    
    connection = sqlite3.connect(os.path.join(os.getcwd(), f'../Database/results.sqlite'))
    cursor = connection.cursor()
    
    insert_records = f'''INSERT INTO {SQLITE_TABLENAME} (category, date, score, count) VALUES(?, ?, ?, ?)
                        ON CONFLICT(category, date) DO 
                        UPDATE SET score = score + excluded.score, count = count + excluded.count 
                        WHERE {SQLITE_TABLENAME}.category LIKE ? AND {SQLITE_TABLENAME}.date LIKE ? '''
    

    contents = []
    for row in rdd.collect():
        contents.append((row[0][0], row[0][1], row[1][0], row[1][1], row[0][0], row[0][1]))

    try:
        cursor.executemany(insert_records, contents)
        connection.commit()
        
    except sqlite3.Error as error:
        print({error})
    finally:
        cursor.close()
        connection.close()

In [ ]:
def update_offset_table():
    """To update the offset values in storage for subsequent data ingestion. """
    
    global OFFSET
    
    connection = sqlite3.connect(os.path.join(os.getcwd(), f'../Database/cache.sqlite'))
    cursor = connection.cursor()

    query = f"UPDATE OFFSET_FINDER SET offsetval = {OFFSET} WHERE topic LIKE ?";
    cursor.execute(query, [TOPIC]);
    connection.commit();
    
    query = f"SELECT offsetval FROM OFFSET_FINDER WHERE topic LIKE ?"
    rows = cursor.execute(query, [TOPIC]).fetchall()

    if rows:
        OFFSET = rows[0][0]
    else:
        OFFSET = -1
        
    print({f"Updated Starting Offset for {TOPIC}": OFFSET})

    cursor.close()
    connection.close()
    
    raise StopExecution

In [26]:
def consumer_call():
    """Consolidated method to handle the Spark processing of data. """
    
    LIMIT_COUNT = 500
    global TABLE_COUNT, OFFSET
    TABLE_COUNT = 1
    # delete_spark_sql_table()
    
    while True:
        query = init_df_table()
        sleep(10)
        
        value = spark.sql(f"SELECT * FROM {IN_MEM_TABLENAME}{TABLE_COUNT}").collect()
        spark.sql(f"DROP TABLE {IN_MEM_TABLENAME}{TABLE_COUNT}")
        
        TABLE_COUNT = (TABLE_COUNT+1)
        OFFSET += len(value)
        
        total_tweet_count = len(value)
        
        print({"Tweets collected from select query": total_tweet_count})
        
        if(total_tweet_count == 0):
            update_offset_table()
        
        iter_count = 0
        
        while len(value):
            
            tweet_dict_list = []
            
            p_bar = tqdm(enumerate(value[:LIMIT_COUNT]))
            
            for indx, row in p_bar:
                jsonCopy = json.loads(row["Data"])
                jsonCopy['score'] = sentiment_score(jsonCopy['tweet'][:135])
                tweet_dict_list.append(jsonCopy)
                p_bar.set_description(f'Working on "{indx + iter_count*LIMIT_COUNT + 1}/{total_tweet_count}"')
                
            print({"Number of tweet records" : len(tweet_dict_list)})
            print("----------------------------------------------------------------")
            query.awaitTermination(1)

            rdd = sc.parallelize(tweet_dict_list)

            newrdd = rdd.map(lambda row: (row['category'], row['date'], row['count'], row['score']))
            newrdd.collect()

            nextrdd = newrdd.map(lambda tup: ((tup[0], tup[1]), (tup[2]*tup[3], tup[2]))).reduceByKey(lambda a, b: (a[0]+b[0], a[1]+b[1]))
            nextrdd.collect()

            write_to_db(nextrdd)

            for i in range(LIMIT_COUNT):
                if(value):
                    value.pop(0)
            
            iter_count += 1   

In [27]:
consumer_call()

root
 |-- Data: string (nullable = true)

+---------+----------+-----------+
|namespace| tableName|isTemporary|
+---------+----------+-----------+
|         |tweetdata1|       true|
+---------+----------+-----------+

{'Tweets collected from select query': 2979}


/usr/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=63, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 43612), raddr=('127.0.0.1', 42903)>
  self._sock = None


0it [00:00, ?it/s]

{'Number of tweet records': 500}
----------------------------------------------------------------


/usr/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=64, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 50588), raddr=('127.0.0.1', 36297)>
  self._sock = None
/usr/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=64, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 57344), raddr=('127.0.0.1', 39681)>
  self._sock = None
/usr/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=65, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 50122), raddr=('127.0.0.1', 38069)>
  self._sock = None


...

0it [00:00, ?it/s]

{'Number of tweet records': 479}
----------------------------------------------------------------


/usr/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=64, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 34150), raddr=('127.0.0.1', 35329)>
  self._sock = None
/usr/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=64, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 34080), raddr=('127.0.0.1', 43901)>
  self._sock = None
/usr/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=65, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 54118), raddr=('127.0.0.1', 39291)>
  self._sock = None


root
 |-- Data: string (nullable = true)

+---------+----------+-----------+
|namespace| tableName|isTemporary|
+---------+----------+-----------+
|         |tweetdata2|       true|
+---------+----------+-----------+

{'Tweets collected from select query': 0}
{'Updated Starting Offset for gaming-tweets': 2979}


/usr/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=64, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 60548), raddr=('127.0.0.1', 39777)>
  self._sock = None


**NOTE**: For re-runs of the program with offset > 0,
cell 19 - 24 (cell that takes limited data from IN_MEM_TABLE, till sqlite3 db connection) - comment out fully, 
cell 25, dont call delete_spark_sql_table()
